# Download Datasets
C:\Users\username\pythonwork\data

In [10]:
import urllib.request
import os #用於確認檔案

if not os.path.exists('data'):
    os.makedirs('data')

url = "https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic3.xls"
filepath = "data/titanic3.xls"
if not os.path.isfile(filepath):
    result = urllib.request.urlretrieve(url, filepath)
    print('downloaded:', result)

# Data preprocessing
需要xlrd讀取Excel文件  
`pip install xlrd`  
使用pandas的read_excel讀取titanic.xls至dataframe


In [11]:
import numpy
import pandas as pd
all_df = pd.read_excel(filepath)
all_df[:2]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"


# Filter datasets

In [12]:
cols = ['survived', 'name', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
all_df = all_df[cols]
all_df[:2]

,survived,name,pclass,sex,age,sibsp,parch,fare,embarked
0,1,"Allen, Miss. Elisabeth Walton",1,female,29.0000,0,0,211.3375,S
1,1,"Allison, Master. Hudson Trevor",1,male,0.9167,1,2,151.5500,S


# pandas dataframe:drop function
`drop(labels = None, axis = 0, index = None, columns = None, inplace = False)`  
 - label就是要刪除的行列名稱 用列表給定
 - axis默認=0 刪除行 因此刪除columns時axis需= 1
 - columns指定要刪除的column
 - inplace = False 默認該刪除操作不改變圓數據 而是return一個執行刪除後的新dataframe
 - inplace = True 則會直接在原數據上做刪除 刪除後無法返回

Reference: [Python中pandas dataframe删除一行或一列：drop函数](https://blog.csdn.net/songyunli1111/article/details/79306639)

In [13]:
df = all_df.drop(['name'], axis = 1)

找出含有null的欄位

In [14]:
all_df.isnull().sum()

survived      0
name          0
pclass        0
sex           0
age         263
sibsp         0
parch         0
fare          1
embarked      2
dtype: int64

fillna:使用指定的方法填充NA/NaN值  
`df.fillna(0)` 將NA/NaN填充為0  
詳細看:[pandas fillna使用](https://blog.csdn.net/conving/article/details/120205513)

In [15]:
# 计算平均年龄和平均票价
age_mean = df['age'].mean()
fare_mean = df['fare'].mean()

# 填充缺失值
df['age'] = df['age'].fillna(age_mean)
df['fare'] = df['fare'].fillna(fare_mean)
df['sex'] = df['sex'].map({'female': 0, 'male': 1}).astype(int, errors='ignore')

x_OneHot_df = pd.get_dummies(data = df, columns=["embarked"])
x_OneHot_df[:2]

,survived,pclass,sex,age,sibsp,parch,fare,embarked_C,embarked_Q,embarked_S
0,1,1,0,29.0000,0,0,211.3375,False,False,True
1,1,1,1,0.9167,1,2,151.5500,False,False,True


In [16]:
ndarray = x_OneHot_df.values
print(ndarray.shape)
print(ndarray[:2])

(1309, 10)
[[1 1 0 29.0 0 0 211.3375 False False True]
 [1 1 1 0.9167 1 2 151.55 False False True]]


In [17]:
Label = ndarray[:, 0]
Features = ndarray[:, 1:]

print(Label[:2])
print(Features[:2])

[1 1]
[[1 0 29.0 0 0 211.3375 False False True]
 [1 1 0.9167 1 2 151.55 False False True]]


# Standardize

In [18]:
from sklearn import preprocessing
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))